In [1]:
import pandas as pd
import numpy as np

In [137]:
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = 5000


In [27]:
house_raw = pd.read_csv('data/train.csv')

In [451]:
house_df.shape

(1460, 420)

In [96]:
house_ty = pd.read_pickle('Ty/house_ty.pkl')
house_bird = pd.read_pickle('Bird/Bird4.pkl')
house_phil = pd.read_pickle('Phil/house_phil.pkl')
house_ray = pd.read_pickle('Ray/house_ray.plk')
house_df = pd.concat([house_ty, house_bird, house_phil, house_ray], axis = 1)

# Get duplicate collumn names 

In [441]:
col_names = house_df.columns

In [444]:
# look at non dummy columns
list(filter(lambda x: not '_' in x, col_names))

In [453]:
import collections
[item for item, count in collections.Counter(original_cols_cleaned).items() if count > 1]

[]

# Fix following columns

In [230]:
dummies = ['SaleType', 'SaleCondition', 'PavedDrive', 'GarageCond', 'GarageQual'] + ["CentralAir", "Electrical", "KitchenQual", "Functional", "FireplaceQu", "GarageType", "GarageFinish"] + ['RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC'] + ['MSSubClass', 'MSZoning', 'Street', 'LotConfig', 'Alley', 'BldgType', 'LotShape', 'LandContour', 'LotConfig', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'LandSlope', 'Neighborhood'] + ['Utilities', 'PoolQC', 'Fence', 'MiscFeature', 'GarageYrBlt'] + ['OverallQual', 'OverallCond'] + ['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces'] + ['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces']

### Add dummies and drop

In [447]:
def add_dummies_remove_modes(dummy_list, house_raw, house_df):

    dummy_modes = list(house_raw[dummy_list].mode().iloc[0,:].items())

    dummy_modes = [(col, (float(mode))) if type(mode) == int else (col, mode) for col, mode in dummy_modes]

    drop_modes = list(map(lambda x: str(x[0]) + '_' + str(x[1]), dummy_modes))

    dummy_cols = pd.get_dummies(house_df[dummy_list].astype(object), dummy_na=True).drop(drop_modes, axis=1)

    return pd.concat([
        house_df.drop(dummy_list, axis=1),
        dummy_cols], axis=1)


### Fill NA for MasVnrArea and LotFrontage

In [377]:
house_df['MasVnrArea'] = house_df['MasVnrArea'].fillna(0)

In [317]:
na_sum = house_df.isna().sum() > 0
house_df.columns[na_sum]

Index(['LotFrontage'], dtype='object')

In [335]:
house_df[['LotFrontage']] = house_df[['LotFrontage']].fillna(0)

# Drop duplicate columns

In [ ]:
cleandf_phil = pd.read_pickle('phil_pickle')

cols_before_drop = set(house_df.columns)
cols_after_drop = set(cleandf_phil.columns)
cols_before_drop - cols_after_drop

In [452]:
cols_before_drop = set(house_df.columns)
cols_after_drop = set(house_df.T.drop_duplicates(keep='first').T.columns)
cols_before_drop - cols_after_drop

{'BedroomAbvGr_nan',
 'BsmtFullBath_nan',
 'BsmtHalfBath_nan',
 'Fireplaces_nan',
 'FullBath_nan',
 'GarageYrBlt_None',
 'GarageYrBlt_nan',
 'HalfBath_nan',
 'KitchenAbvGr_nan',
 'TotRmsAbvGrd_14.0',
 'TotRmsAbvGrd_nan'}

In [ ]:
house_df = house_df.T.drop_duplicates(keep='first').T

# Random Forest

In [374]:
house_df = pd.read_pickle('train.pkl')

In [375]:
X = house_df.drop('SalePrice',axis=1)
y = house_df.SalePrice

In [376]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000, max_features=3, oob_score=True)
rf.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=3, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=None, oob_score=True,
           random_state=None, verbose=0, warm_start=False)

In [351]:
rf.oob_score_

0.7994944202295384

In [350]:
pd.DataFrame([X.columns, rf.feature_importances_]).T.sort_values(1, ascending=False)

,0,1
1,GarageArea,0.0341169
0,GarageCars,0.0334087
243,GrLivArea,0.0331679
240,1stFlrSF,0.0308198
126,TotalBsmtSF,0.0302505
38,YearBuilt,0.0279526
123,BsmtFinSF1,0.0240092
246,FullBath,0.0237053
39,YearRemodAdd,0.0224641
37,LotArea,0.0222535
